In [1]:
!pip install BeautifulSoup4 openpyxl unidecode scikit-learn requests selenium
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 48 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of addi

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
## Comenzamos analizando el formato de la url en la cual podemos consultar de manera normal la data
'''
URl para obtener inmuebles de todas las alcaldias
https://century21mexico.com/v/resultados/tipo_casa-o-casa-duplex-'
o-casa-en-condominio-o-departamento-o-penthouse-o-departamento-cuadruplex-
o-departamento-triplex/operacion_venta/en-pais_mexico/en-estado_ciudad-de-mexico
/en-municipio_ciudad-de-mexico-alvaro-obregon-o-azcapotzalco-
o-ciudad-de-mexico-benito-juarez-o-ciudad-de-mexico-coyoacan-o-cuajimalpa-de-morelos-
o-ciudad-de-mexico-cuauhtemoc-o-gustavo-a.-madero-o-iztacalco-o-iztapalapa-
o-ciudad-de-mexico-la-magdalena-contreras-o-miguel-hidalgo-o-milpa-alta-
o-tlalpan-o-tlahuac-o-ciudad-de-mexico-venustiano-carranza-o-xochimilco'''

'''
Url para obtener inmuebles de todo cdmx
https://century21mexico.com/v/resultados/tipo_casa-o-casa-duplex-o-casa-en-condominio-o-town-house/operacion_venta/en-pais_mexico/en-estado_ciudad-de-mexico/
'''

base_url = 'https://century21mexico.com'

## Podemos observar que la peticion se construye por la direccion principal
base_consult_url = f'{base_url}/v/resultados/'

## Lo que sigue en la url es ahora el tipo de casa el cual esta separado por letras 'o' y concatenado con '-'
## por lo cual podemos armarlo de manera estandarizada
property_type = [
    'casa duplex',
    'casa en condominio',
    'departamento',
    'penthouse',
    'departamento cuadruplex',
    'departamento triplex'
]
## Ahora le damos formato para cambiar los espacios por guiones
property_type = [tipo.replace(' ', '-') for tipo in property_type]
print(property_type)

## El tipo de operacion podemos elegir tanto venta como compra
operation = 'venta'
print(operation)

## Selecionamos ahora el pais y el estado
country = 'mexico'
state = 'ciudad de mexico'
state = state.replace(' ','-')
print(country, '\n', state)

## Ahora construimos nuestra url
url = f"{base_consult_url}tipo_casa-o-{'-o-'.join(property_type)}/operacion_{operation}/en-pais_{country}/en-estado_{state}"
print(f'Url completa: {url}')

['casa-duplex', 'casa-en-condominio', 'departamento', 'penthouse', 'departamento-cuadruplex', 'departamento-triplex']
venta
mexico 
 ciudad-de-mexico
Url completa: https://century21mexico.com/v/resultados/tipo_casa-o-casa-duplex-o-casa-en-condominio-o-departamento-o-penthouse-o-departamento-cuadruplex-o-departamento-triplex/operacion_venta/en-pais_mexico/en-estado_ciudad-de-mexico


In [4]:
## Ahora comenzamos a explorar el contenido que nos devuelve la peticion a nuestra url
response = requests.get(url)
page = BeautifulSoup(response.text)
page ## Podemos observar que no podemos proceder solamente con requests por que el contenido se carga dinamicamente

<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="- México" name="author"/>
<title>- México | Bienes Raíces, Casas y Departamentos en venta y renta, Asesores y Asesoría inmobiliaria</title>
<link href="https://fonts.googleapis.com" rel="preconnect"/>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;700&amp;display=swap" rel="stylesheet"/>
<style type="text/css">
			.gray{
				color: #bcbcbc!important;
			}
			.white{
				color: #fff!important;
			}
			.black{
				color: #000!important;
			}
			
			/*END colors*/
			
			#map-static {
				width: 100%;
				height: 300px;
				overflow: hidden;
				position: relative;
				cursor: pointer;
				display: block;
			}
			#map-static .img-static-map {
				min-height: 100%;
				max-width: 100%;
				min-width: 100%;
				position: absolute;
		

In [5]:
## Al no tener conocimiento pleno de selenium optaremos por conseguir primero las paginas, ya que explorando
## podemos observar que la peticion agrega solo al final el numero de pagina a consultar

## Por lo cual procederemos a utilizar selenium webdriver
# Primero
# Crear una instancia del navegador
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
driver.get(url)

pages = set()
try:
    time.sleep(5)
    while True:
        # Realizar alguna acción en la página actual
        #elementos = driver.find_elements(By.CLASS_NAME, "d-flex")
        #for elemento in elementos:
        #    print(elemento.text)

        # Intentar encontrar el botón de siguiente página
        try:
            paginator = driver.find_elements(By.XPATH, "//li[@class='page-item' or @class='page-item active']")
            for i in paginator:
                pages.add(i.text)

            next_button = driver.find_element(By.XPATH, "//li[@class='page-item']/a[@class='page-link' and contains(span, '›')]")
            next_button.click()
            time.sleep(5)

        except:
            print("No hay más páginas.")
            break
finally:
    # Cerrar el navegador
    driver.quit()
pagesInt = [int(i) for i in pages if i.isdigit()]
max_page = max(pagesInt)
min_page = min(pagesInt)
print(f'min={min_page} max={max_page}')

No hay más páginas.
min=1 max=15


In [6]:
## Y ahora que contamos con todas las paginas que cuentan con datos extraeremos todos los elementos
## de cada pagina

## Construimos nuestras nuevas urls
urls = [url + f'/pagina_{str(i)}' for i in range(min_page, max_page + 1)]
urls

['https://century21mexico.com/v/resultados/tipo_casa-o-casa-duplex-o-casa-en-condominio-o-departamento-o-penthouse-o-departamento-cuadruplex-o-departamento-triplex/operacion_venta/en-pais_mexico/en-estado_ciudad-de-mexico/pagina_1',
 'https://century21mexico.com/v/resultados/tipo_casa-o-casa-duplex-o-casa-en-condominio-o-departamento-o-penthouse-o-departamento-cuadruplex-o-departamento-triplex/operacion_venta/en-pais_mexico/en-estado_ciudad-de-mexico/pagina_2',
 'https://century21mexico.com/v/resultados/tipo_casa-o-casa-duplex-o-casa-en-condominio-o-departamento-o-penthouse-o-departamento-cuadruplex-o-departamento-triplex/operacion_venta/en-pais_mexico/en-estado_ciudad-de-mexico/pagina_3',
 'https://century21mexico.com/v/resultados/tipo_casa-o-casa-duplex-o-casa-en-condominio-o-departamento-o-penthouse-o-departamento-cuadruplex-o-departamento-triplex/operacion_venta/en-pais_mexico/en-estado_ciudad-de-mexico/pagina_4',
 'https://century21mexico.com/v/resultados/tipo_casa-o-casa-duplex-o

In [7]:
## Extraemos el codigo de todas las paginas
pageCode = []
# Crear una instancia del navegador
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
## Extraemos
try:
    time.sleep(6)
    for i in urls:
        driver.get(i)
        time.sleep(6)
        # Realizar alguna acción en la página actual
        e = driver.page_source
        pageCode.append(e)
finally:
    # Cerrar el navegador
    driver.quit()

In [8]:
## Obtenemos ahora cada propiedad
elements = []
for i in pageCode:
    aux = BeautifulSoup(i).find_all('a', class_='text-decoration-none h5 link-dark text-truncate ucfirst mb-0 stretched-link')
    ## agregamos solo el href sobre la base_url para que resulte una url de esta forma
    ## https://century21mexico.com/propiedad/548198_venta-departamento-en-paseos-de-taxquena
    elements = elements + [base_url+x['href'] for x in aux]
len(elements)

1424

In [10]:
elements[1423]

'https://century21mexico.com/propiedad/305626_casa-en-venta-col-panamericana-gustavo-a-madero-cdmx'

In [11]:
## Ahora si extraemos toda la informacion que necesitamos
properties = []
# Crear una instancia del navegador
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
## Extraemos propiedad por propiedad
try:
    time.sleep(2)
    for i in elements:
        driver.get(i)
        time.sleep(2)
        # Realizar alguna acción en la página actual
        e = driver.page_source
        properties.append((i,e))
finally:
    # Cerrar el navegador
    driver.quit()

In [12]:
# Comprobamos la cantidad de inmuebles que conseguimos
len(properties)

1424

In [13]:
pruebaBs = BeautifulSoup(properties[0][1])

In [14]:
# Definimos una funcion donde extraeremos los datos de cada pagina
def extract_data(soup):
    terreno = None
    construccion = None
    recamaras = None
    banos = None
    estacionamientos = None
    ubicacion = None
    precio = None
    anio_construct = None
    tipo = None
    # Extraemos m2 de terreno y construccion, no. de recamaras, baños y estacionamientos
    for i in soup.find_all('div',class_='col my-2'):
        if 'Terreno' in i.text:
            terreno = i.text.split(' ')[1].strip()
        elif 'Construcción' in i.text:
            construccion = i.text.split(' ')[1].strip()
        elif 'Recámaras' in i.text:
            recamaras = i.text.split(' ')[1].strip()
        elif 'Baños' in i.text:
            banos = i.text.split(' ')[1].strip()
        elif 'Estacionamientos' in i.text:
            estacionamientos = i.text.split(' ')[1].strip()
    # Extraemos el ultimo parrafo formateado con la clase text-muted que es la ubicacion
    ubicacion = soup.find_all('p',class_='text-muted')[-1].text
    # Extraemos el precio que encontramos en negritas al principio de la pagina
    # debajo de las imagenes y titulo
    precio = soup.find('h6',class_='text-muted fs-3 fw-bold').text
    # Extraemos el anio de construccion y el tipo de inmueble
    for i in soup.find_all('div',class_='col-sm-12 col-md-6 my-1'):
        if 'Año de Construcción' in i.text:
            anio_construct = i.text.split(':')[1].strip()
        elif 'Tipo' in i.text:
            tipo = i.text.split(':')[1].strip()
    # En algunos casos no viene la construccion o el terreno pero si el otro
    # para eso agregamos un if
    if (terreno is None or terreno == '0.0') and construccion is not None:
        terreno = construccion
    elif (construccion is None or construccion == '0.0') and terreno is not None:
        construccion = terreno
    # Tambien si no encuentra el estacionamiento entonces los dejamos en cero
    if estacionamientos is None:
        estacionamientos = 0
    return {'terreno':terreno,'construccion':construccion,"recamaras":recamaras,"banos":banos,
            "estacionamientos":estacionamientos,"ubicacion":ubicacion,"precio":precio,
            "anio_construct":anio_construct,"tipo":tipo}

In [15]:
# Probamos que funciona
extract_data(pruebaBs)

{'terreno': '61.5',
 'construccion': '61.5',
 'recamaras': '2',
 'banos': '1',
 'estacionamientos': '1',
 'ubicacion': 'Doctores, Cuauhtémoc, Ciudad De México 06720, México.',
 'precio': '$2,900,000 MXN',
 'anio_construct': '2022',
 'tipo': 'regular'}

In [16]:
columns_df=['terreno_m2','construccion_m2','recamaras','baños',
            'estacionamientos','ubicacion','precio',
            'anio_construccion','tipo']

In [18]:
# Ahora iteramos sobre todas nuestras propiedades para extraer sus datos
data = []
errors = []
for i in properties:
    try:
        # Agregamos a la lista data cada lista de datos extraidos
        # pasamos
        data.append(extract_data(BeautifulSoup(i[1])))
    except Exception as e:
        print(f'Error extrayendo data de: {i[0]}\n {e}')
        errors.append(i)
        continue

Error extrayendo data de: https://century21mexico.com/propiedad/546205_casa-en-venta-colonia-valle-escondido-tlalpan-ciudad-de-mexico
 list index out of range
Error extrayendo data de: https://century21mexico.com/propiedad/492218_venta-de-hermosa-residencia-en-santa-fe
 list index out of range
Error extrayendo data de: https://century21mexico.com/propiedad/519229_excelente-departamento-completamente-nuevo
 list index out of range
Error extrayendo data de: https://century21mexico.com/propiedad/519235_departamento-en-venta-completamente-nuevo
 list index out of range
Error extrayendo data de: https://century21mexico.com/propiedad/475519_departamento-en-venta-col-acueducto-de-guadalupe
 list index out of range
Error extrayendo data de: https://century21mexico.com/propiedad/468710_departamento-en-venta-en-colonia-san-agustin-pueblo-de-tetelpan-ciudad-de-mexico
 list index out of range
Error extrayendo data de: https://century21mexico.com/propiedad/468641_del-carmen-coyoacan
 list index out

In [22]:
# Revisando los links con errores podemos ver que ya no existen o estan terminados
# Por lo cual podemos descartarlos y nos quedarian 1400 propiedades para analizar
print(f"Correctos:{len(data)}, errores:{len(errors)}")

Correctos:1416, errores:8


In [25]:
# Reviamos nuestra data
pd.DataFrame(data).sample(random_state=15, n=10)

,terreno,construccion,recamaras,banos,estacionamientos,ubicacion,precio,anio_construct,tipo
816,"1,347.0",76.0,None,None,100,"Los Alpes, Alvaro Obregón, Ciudad De México 01...","$5,692,464 MXN",2023,regular
218,51.2,86.1,2,2,0,"Unidad Santa Fe IMSS, Alvaro Obregón, Ciudad D...","$2,500,000 MXN",1974,casa-en-condominio
38,79.0,79.0,2,2,2,"Vértiz Narvarte, Benito Juárez, Ciudad De Méxi...","$3,990,000 MXN",12,laminado
469,96.2,96.2,3,2,2,"Miguel Hidalgo, Tlalpan, Ciudad De México 1425...","$3,700,000 MXN",10,Loseta
704,58.0,58.0,2,2,0,"San Miguel Amantla, Azcapotzalco, Ciudad De Mé...","$2,230,000 MXN",2024,departamento
1186,336.0,566.0,3,4,4,"Jardines Del Pedregal, Alvaro Obregón, Ciudad ...","$34,000,000 MXN",2016,regular
270,103.5,65.0,1,2,1,"Nativitas, Benito Juárez, Ciudad De México 035...","$4,454,000 MXN",2024,departamento
335,74.0,74.0,2,2,1,"Santa Maria Nonoalco, Benito Juárez, Ciudad De...","$4,495,000 MXN",2024,departamento
806,"1,411.0",34.0,1,1,30,"Roma Norte, Cuauhtémoc, Ciudad De México 06700...","$3,509,376 MXN",2023,regular
1385,84.2,84.2,2,2,2,"Residencial Zacatenco, Gustavo A. Madero, Ciud...","$4,190,000 MXN",2021,Porcelanato y Cerámica


In [28]:
pd.DataFrame(data).describe()

,terreno,construccion,recamaras,banos,estacionamientos,ubicacion,precio,anio_construct,tipo
count,1416,1416,1371,1382,1416,1416,1416,1390,1416
unique,618,644,14,12,20,521,717,136,138
top,200.0,64.0,2,2,2,"Del Valle, Benito Juárez, Ciudad De México 031...","$6,000,000 MXN",2023,departamento
freq,23,15,500,560,435,40,18,101,406


In [29]:
# Guardamos nuestros datos extraidos para no estar siempre extrayendo, porque el tiempo
# para ejecutar promedio es de 2 hrs para llegar hasta este punto
pd.DataFrame(data).to_csv('data_propiedades.csv', index=False, header=True)